In [6]:
!pip install youtokentome OpenNMT-py pymysql coloredlogs

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import logging

import coloredlogs

from run import run

coloredlogs.install(level="WARNING")
logging.getLogger("matplotlib.axes._base").setLevel(logging.INFO)

In [10]:
# How to launch gitbase
gitbase_index_dir = "/ssd/devfest2019-workshop/repos/index"
bblfsh = "bblfshd:9432"
repos_loc = "/ssd/devfest2019-workshop/repos/repos"
"""
GITBASE_UAST_CACHE_SIZE=0 BBLFSH_ENDPOINT=%s GITBASE_INDEX_DIR=%s ./gitbase server -v -d %s
""" % (bblfsh, gitbase_index_dir, repos_loc)

'\nGITBASE_UAST_CACHE_SIZE=0 BBLFSH_ENDPOINT=bblfshd:9432 GITBASE_INDEX_DIR=/ssd/devfest2019-workshop/repos/index ./gitbase server -v -d /ssd/devfest2019-workshop/repos/repos\n'

In [11]:
function_group = """SELECT
    files.repository_id as repository_id,
    files.file_path as path,
    files.blob_content as content,
    UAST(files.blob_content, LANGUAGE(files.file_path, files.blob_content), '//uast:FunctionGroup') as functions
FROM files
NATURAL JOIN commit_files
NATURAL JOIN commits
NATURAL JOIN refs
WHERE
    refs.ref_name= 'HEAD' and functions IS NOT NULL
    AND LANGUAGE(files.file_path,files.blob_content) = 'Java'
    AND NOT IS_VENDOR(file_path)
    AND NOT IS_BINARY(file_path)
"""


function_identifiers = """SELECT
    count(*)

FROM (%s) as f

""" % (function_group)

print(function_identifiers)
run(function_identifiers)

SELECT
    count(*)

FROM (SELECT
    files.repository_id as repository_id,
    files.file_path as path,
    files.blob_content as content,
    UAST(files.blob_content, LANGUAGE(files.file_path, files.blob_content), '//uast:FunctionGroup') as functions
FROM files
NATURAL JOIN commit_files
NATURAL JOIN commits
NATURAL JOIN refs
WHERE
    refs.ref_name= 'HEAD' and functions IS NOT NULL
    AND LANGUAGE(files.file_path,files.blob_content) = 'Java'
    AND NOT IS_VENDOR(file_path)
    AND NOT IS_BINARY(file_path)
) as f




/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (0, 'unable to parse the given blob using bblfsh: syntax error: Syntax error on token(s), misplaced construct(s)\nSyntax error on token "module", package expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSy

(21385,)

# TODO - supress warnings

# Select function definitions

In [2]:
function_group = """SELECT
    files.repository_id as repository_id,
    files.file_path as path,
    files.blob_content as content,
    UAST(files.blob_content, LANGUAGE(files.file_path, files.blob_content), '//uast:FunctionGroup') as functions
FROM files
NATURAL JOIN commit_files
NATURAL JOIN commits
NATURAL JOIN refs
WHERE
    refs.ref_name= 'HEAD' and functions IS NOT NULL
    AND LANGUAGE(files.file_path,files.blob_content) = 'Java'
    AND NOT IS_VENDOR(file_path)
    AND NOT IS_BINARY(file_path)
"""

repositories, paths, contents, function_groups = run(function_group)

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (0, 'unable to parse the given blob using bblfsh: syntax error: Syntax error on token(s), misplaced construct(s)\nSyntax error on token "module", package expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSyntax error on token ".", = expected\nSyntax error on token(s), misplaced construct(s)\nSy

In [3]:
len(repositories)

21385

In [ ]:
# TODO: deduplicate functions

# Extract function names & identifiers from the body

In [4]:
DEFAULT = "\033[30m"
WHITE = "\033[0m"
RED = "\033[31m"
GREEN = "\033[32m"
ORANGE = "\033[33m"
BLUE = "\033[34m"
PURPLE = "\033[35m"


def colored_text_by_pos(text, colored_pos):
    if not colored_pos:
        return text
    res_text = []
    col_pos = []
    
    colored_pos_ = sorted(colored_pos, key=lambda x: x.pos)
    
    if colored_pos_[0].pos[0] != 0:
        # add text before first colored text
        col_pos.append(Colored(color=DEFAULT, position=(0, colored_pos_[0].pos[0])))
    if len(colored_pos_) > 1:
        for first, second in zip(colored_pos_[:-1], colored_pos_[1:]):
            col_pos.append(first)
            col_pos.append(Colored(color=DEFAULT, position=(first.pos[1], second.pos[0])))
    col_pos.append(colored_pos_[-1])
    col_pos.append(Colored(color=DEFAULT, position=(colored_pos_[-1].pos[1], len(text))))
    
    for cp in col_pos:
        res_text.append(cp.color + text[cp.pos[0]:cp.pos[1]])
    return "".join(res_text)

In [5]:
import pprint

from bblfsh.pyuast import decode as uast_decode
from tqdm import tqdm_notebook as tqdm


class Colored:
    def __init__(self, color, position, start_offset=0):
        self.color = color
        self.pos = tuple(map(lambda x: x - start_offset, position))



def get_function_name(function_node):
    func_name, func_name_pos = None, None
    for node in function_node["Nodes"]:
        if node is None or "@type" not in node:
            continue
        if node["@type"] == 'uast:Alias':
            func_name = node["Name"]["Name"]
            func_name_pos = (node["Name"]["@pos"]["start"]["offset"], node["Name"]["@pos"]["end"]["offset"])
    return func_name, func_name_pos


def traverse_tree_yield_identifiers(node):
    if (isinstance(node, dict) and 
        '@type' in node and 
        node['@type']  == 'uast:Identifier'):
        yield node["Name"], (node["@pos"]["start"]["offset"], node["@pos"]["end"]["offset"])
    else:
        if isinstance(node, dict):
            for k in node:
                yield from traverse_tree_yield_identifiers(node[k])
        elif isinstance(node, list) or isinstance(node, tuple):
            for n in node:
                yield from traverse_tree_yield_identifiers(n)

In [6]:
# show several cases
for content, func_group in zip(contents[:3], function_groups[:3]):
    func_group = uast_decode(func_group, format=0).load()
    content = content.decode()
    
    for func in func_group:
        identifiers = None
        for node in func["Nodes"]:        
            if node is None or "@type" not in node:
                continue
            if node["@type"] == 'uast:Alias':
                func_body = node["Node"]["Body"]
            # select identifiers from function body
                body_identifiers = sorted(traverse_tree_yield_identifiers(func_body), key=lambda x: x[1][0])
        func_name = get_function_name(func)
    #     print("-" * 20, func_name, ":", "|".join(map(str, body_identifiers)), "\n\n")
        print("-" * 20)

        start_offset = func["@pos"]["start"]["offset"]
        colored_texts = []
        colored_texts.append(Colored(color=RED, position=func_name[1], start_offset=start_offset))
        for bi in body_identifiers:
            colored_texts.append(Colored(color=GREEN, position=bi[1], start_offset=start_offset))

        print(colored_text_by_pos(content[start_offset:func["@pos"]["end"]["offset"]], colored_texts))

--------------------
public static void main(String[] args) {
        System.out.println(getAddress());
    }
--------------------
public static String getAddress() {
        return getLocalAddress().getHostAddress();
    }
--------------------
private static InetAddress getLocalAddress() {
        InetAddress localAddress = null;
        try {
            localAddress = InetAddress.getLocalHost();
            Optional<InetAddress> addressOp = toValidAddress(localAddress);
            if (addressOp.isPresent()) {
                return addressOp.get();
            }
        } catch (Throwable e) {
            e.printStackTrace();
        }

        try {
            Enumeration<NetworkInterface> interfaces = NetworkInterface.getNetworkInterfaces();
            if (null == interfaces) {
                return localAddress;
            }
            while (interfaces.hasMoreElements()) {
                try {
                    NetworkInterface network = interfaces.nextElement();
      

# TODO: fix it
```---------------------------------------------------------------------------
UnicodeDecodeError                        Traceback (most recent call last)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbc in position 0: invalid start byte

The above exception was the direct cause of the following exception:

SystemError                               Traceback (most recent call last)
SystemError: <class 'UnicodeDecodeError'> returned a result with an error set

The above exception was the direct cause of the following exception:

SystemError                               Traceback (most recent call last)
<ipython-input-21-e92edd4f6a41> in <module>
      3 function_names = []
      4 for content, func_group in tqdm(zip(contents, function_groups), total=len(contents)):
----> 5     func_group = uast_decode(func_group, format=0).load()
      6     content = content.decode()
      7 

SystemError: <built-in method load of pyuast.ContextExt object at 0x7f9e68138b50> returned a result with an error set
```

In [9]:
# prepare raw X (identifiers from the body) & y (name of the function)
from joblib import Parallel, delayed

def process_function_group(func_group):
    res = []
    try:
        func_group = uast_decode(func_group, format=0).load()


        for func in func_group:
            body_identifiers = None        
            for node in func["Nodes"]:        
                if node is None or "@type" not in node:
                    continue
                if node["@type"] == 'uast:Alias':
                    func_body = node["Node"]["Body"]
                    # select identifiers from function body
                    body_identifiers = sorted(traverse_tree_yield_identifiers(func_body), key=lambda x: x[1][0])
            if body_identifiers is None:
                continue
            res.append(([i[0] for i in body_identifiers], get_function_name(func)[0]))
    except:
        print("decoding error")
    return res

function_group_res = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_function_group)(fg) for fg in function_groups)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1452s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 211 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 253 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 294 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 339 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 385 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 435 ta

In [20]:
import itertools
res = list(itertools.chain.from_iterable(function_group_res))
deduplicated_res = list(filter(lambda x: x[0], set(map(lambda x: (tuple(x[0]), x[1]), res))))
len(deduplicated_res)

139696

# Prepare BPE input & train BPE

In [27]:
# train BPE

bodies = []
names = []
text_file = "/ssd/devfest2019-workshop/bpe_input.txt"
with open(text_file, "wt") as f:
    for p in tqdm(deduplicated_res):
        bodies.append(" ".join(p[0]))
        names.append(p[1])
        
        f.write(bodies[-1] + " " + names[-1] + "\n")

bpe_model_loc = "/ssd/devfest2019-workshop/bpe.model"
vocab_size = 10000
!yttm bpe --data {text_file} --model {bpe_model_loc} --vocab_size {vocab_size}


Training parameters
  input: /ssd/devfest2019-workshop/bpe_input.txt
  model: /ssd/devfest2019-workshop/bpe.model
  vocab_size: 10000
  n_threads: 8
  character_coverage: 1
  pad: 0
  unk: 1
  bos: 2
  eos: 3

reading file...
learning bpe...
number of unique characters in the training data: 64
number of deleted characters: 0
number of unique characters left: 64
id: 1000=654+23               freq: 3897        subword: ▁any=▁an+y
id: 2000=1960+125             freq: 1355        subword: emplate=empl+ate
id: 3000=869+2852             freq: 714         subword: ▁DEBUG=▁DE+BUG
id: 4000=85+35                freq: 461         subword: lex=le+x
id: 5000=774+701              freq: 328         subword: ▁parseLong=▁parse+Long
id: 6000=1339+1531            freq: 249         subword: ▁hostname=▁host+name
id: 7000=409+5151             freq: 198         subword: INTERVAL=IN+TERVAL
id: 8000=6311+3259            freq: 162         subword: ▁VARCHAR=▁VAR+CHAR
id: 9000=1032+448             freq: 136      

# Train/test split + prepare vocabulary + encode text to have ids that will be fed to NN

In [28]:
from sklearn.model_selection import train_test_split

train_bodies, val_bodies, train_names, val_names = train_test_split(bodies, names, 
                                                                    test_size=0.1, random_state=1989)

In [29]:
train_bodies_loc = "/ssd/devfest2019-workshop/text_input/train.src"
with open(train_bodies_loc, "wt") as f:
    for p in tqdm(train_bodies):        
        f.write(p + "\n")
train_names_loc = "/ssd/devfest2019-workshop/text_input/train.tgt"
with open(train_names_loc, "wt") as f:
    for p in tqdm(train_names):        
        f.write(p + "\n")
        
val_bodies_loc = "/ssd/devfest2019-workshop/text_input/val.src"
with open(val_bodies_loc, "wt") as f:
    for p in tqdm(val_bodies):        
        f.write(p + "\n")
val_names_loc = "/ssd/devfest2019-workshop/text_input/val.tgt"
with open(val_names_loc, "wt") as f:
    for p in tqdm(val_names):        
        f.write(p + "\n")


In [39]:
source_vocab = "/ssd/devfest2019-workshop/text_input/src.vocab"
with open(source_vocab, "w") as f:
    for i in range(vocab_size + 5):
        f.write(str(i) + "\n")

target_vocab = "/ssd/devfest2019-workshop/text_input/tgt.vocab"
with open(target_vocab, "w") as f:
    for i in range(vocab_size + 5):
        f.write(str(i) + "\n")

In [32]:
import youtokentome as yttm

bpe = yttm.BPE(model=bpe_model_loc)

In [34]:
bpe

In [35]:
train_bodies_bpe_loc = "/ssd/devfest2019-workshop/bpe_input/train.src"
with open(train_bodies_bpe_loc, "w") as f:
    for row in bpe.encode(train_bodies, output_type=yttm.OutputType.ID):
        f.write(" ".join(map(str, row)) + "\n")

train_names_bpe_loc = "/ssd/devfest2019-workshop/bpe_input/train.tgt"
with open(train_names_bpe_loc, "w") as f:
    for row in bpe.encode(train_names, output_type=yttm.OutputType.ID):
        f.write(" ".join(map(str, row)) + "\n")
        
val_bodies_bpe_loc = "/ssd/devfest2019-workshop/bpe_input/val.src"
with open(val_bodies_bpe_loc, "w") as f:
    for row in bpe.encode(val_bodies, output_type=yttm.OutputType.ID):
        f.write(" ".join(map(str, row)) + "\n")
val_names_bpe_loc = "/ssd/devfest2019-workshop/bpe_input/val.tgt"
with open(val_names_bpe_loc, "w") as f:
    for row in bpe.encode(val_names, output_type=yttm.OutputType.ID):
        f.write(" ".join(map(str, row)) + "\n")


# Train transformer

In [ ]:
"""-data /ssd/devfest2019-workshop/opennmt_format_input -save_model /ssd/devfest2019-workshop/transformer_bpe \
        -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
        -encoder_type transformer -decoder_type transformer -position_encoding \
        -train_steps 200000  -max_generator_batches 2 -dropout 0.1 \
        -batch_size 4096 -batch_type tokens -normalization tokens  -accum_count 2 \
        -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 \
        -max_grad_norm 0 -param_init 0  -param_init_glorot \
        -label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 10000 \
        -world_size 4 -gpu_ranks 0 1 2 3 """

In [36]:
# preprocess
!echo -train_src {train_bodies_bpe_loc} \
    -train_tgt {train_names_bpe_loc} \
    -valid_src {val_bodies_bpe_loc} \
    -valid_tgt {val_names_bpe_loc} \
    -save_data /ssd/devfest2019-workshop/opennmt_format_input

-train_src /ssd/devfest2019-workshop/bpe_input/train.src -train_tgt /ssd/devfest2019-workshop/bpe_input/train.tgt -valid_src /ssd/devfest2019-workshop/bpe_input/val.src -valid_tgt /ssd/devfest2019-workshop/bpe_input/val.tgt -save_data /ssd/devfest2019-workshop/opennmt_format_input


# Train transformer with openNMT-tf

In [ ]:
yaml_content = """
model_dir: run/

data:
  train_features_file: text_input/train.src
  train_labels_file: text_input/train.tgt
  eval_features_file: text_input/val.src
  eval_labels_file: text_input/val.tgt
  source_vocabulary: text_input/src.vocab
  target_vocabulary: text_input/tgt.vocab
"""

In [ ]:
cmd = "CUDA_VISIBLE_DEVICES=0,1,2,3 onmt-main --model_type Transformer --config openNMT_tf_train_data.yml \
--auto_config train --with_eval --num_gpus 4"